In [17]:
import json
import datetime
import os
import copy
import random 
import numpy as np
from client import *
import matplotlib.pyplot as plt

In [18]:
Team_Key = "VPN75t7UGO44XsrQlo9KuKkg8yRrqgSfyh13GITsQnyvLLyvV5" 
Team_Name = "Platypus_Perry"
max_limit = 10 
min_limit = -10
generations = 10
array_len = 11
population_size = 8
chromosome_size = 11
Mutation_Probability = 0.8         # probabilty that it will mutate
Mutation_Difference_Scale = 1000   # adding noice on mutation to actual value ratio
initial_array = [0.0, -1.45799022e-12, -2.28980078e-13,  4.62010753e-11, -1.75214813e-10, -1.83669770e-15,  8.52944060e-16,  2.29423303e-05, -2.04721003e-06, -1.59792834e-08,  9.98214034e-10]
valid_err_best_weights = 0
train_err_best_weights = 0
best_weights_set = []
fitness_best_weights = 0
population = []

In [19]:
def add_noise(arr):
    for id, val in np.ndenumerate(arr):
        Mutation_Probability = random.uniform(0.2,0.8)
        Mutation_Difference_Scale = np.random.randint(100, 700)
        if np.random.random() < Mutation_Probability:
            range_var = val/Mutation_Difference_Scale
            arr[id] = arr[id] + np.random.uniform(-range_var,range_var)
    return np.clip(arr, min_limit, max_limit)

In [20]:
def populate(arr):
    arra = []
    for i in range(0, population_size):
        arra.append(arr)
    arra = np.array(arra, dtype=np.double)
    arra = add_noise(arra)
    arra[0] = arr
    return arra

In [21]:
def Roulette_Wheel_Selection(arr ,fitness_sum):
    random_nu = random.uniform(0,fitness_sum)
    for id, val in np.ndenumerate(arr):
        if (val > random_nu):
            return id
    return len(arr) - 1 

In [22]:
def get_gen(name_file ,i , population):
    j = i
    selected_population = []
    crossovered_population = []
    mutated_population = []
    # global population
    global fitness_best_weights
    global valid_err_best_weights
    global train_err_best_weights
    #finding errors
    fitness = []
    train_error = []
    valid_error = []
    for chromosomes in population:
        # fitness.append(get_fake_error(list(chromosomes)))

        train_err, valid_err = get_errors(Team_Key, list(chromosomes))
        # train_err = 100 - random.uniform(0,5*j)
        # valid_err = 100 - random.uniform(0,5*j)
        train_error.append(train_err)
        valid_error.append(valid_err)
        fitness.append(-(train_err + valid_err))

    #updating err values
    best_fit_id = np.argmax(fitness)
    if ( (fitness_best_weights == 0) or (fitness[best_fit_id] > fitness_best_weights)):
        fitness_best_weights = fitness[best_fit_id]
        valid_err_best_weights = valid_error[best_fit_id]
        train_err_best_weights = train_error[best_fit_id]
        best_weights_set[:] = population[best_fit_id]

    scaled_fitness = (fitness - np.min(fitness)) / np.ptp(fitness) #fitness is scaled from 0 to 1
    partial_sum = []
    sum_fitness = np.sum(scaled_fitness)
    part = 0.0
    for val in scaled_fitness:
        part = part + val
        partial_sum.append(part)

    crossover =[]
    for i in range(0, population_size//2):
        # selection
        selected_f = population[Roulette_Wheel_Selection(partial_sum ,sum_fitness)]
        selected_s = population[Roulette_Wheel_Selection(partial_sum ,sum_fitness)]
        selected_population.append(selected_f)
        selected_population.append(selected_s)

        # crossover 
        crossovered_f = np.copy(selected_f)
        crossovered_s = np.copy(selected_s)
        crossover_point = np.random.randint(chromosome_size)
        crossovered_f[0:crossover_point] = selected_f[0:crossover_point]
        crossovered_s[0:crossover_point] = selected_s[0:crossover_point]
        crossovered_population.append(crossovered_f)
        crossovered_population.append(crossovered_s)
        crossover.append(crossovered_f)
        crossover.append(crossovered_s)
    
    #mutation
    crossover = np.array(crossover, dtype=np.double)
    population = add_noise(crossover)
    mutated_population = population

    #creating generation files
    file_name = str(name_file) + "/" + "generations_" + str(j+1) + ".txt"
    
    selected_population = np.array(selected_population)
    crossovered_population = np.array(crossovered_population)
    mutated_population = np.array(mutated_population)
    with open(file_name, 'w') as write_file:
        json.dump(selected_population.tolist(), write_file)
        write_file.write('\n' + '\n')
        json.dump(crossovered_population.tolist(), write_file)
        write_file.write('\n' + '\n')
        json.dump(mutated_population.tolist(), write_file)
    return population


In [23]:
name_file = "generations22"
if (os.path.isdir(name_file) != 1):
    os.mkdir(name_file)
population = populate(initial_array)
population = np.array(population , dtype=np.double)

for i in range(0,generations):
    population = get_gen(name_file, i , population)
    
print("best values >>>>>>>>",name_file,datetime.datetime.now(),file=open("tt.txt" , "a"))
print(fitness_best_weights,"\n", valid_err_best_weights,"\n", train_err_best_weights,"\n", best_weights_set,file=open("tt.txt" , "a"))
print("=================================================================================\n\n\n\n",file=open("tt.txt" , "a"))


In [24]:
vvtt = [0.0, -1.4596024506458101e-12, -2.294861324029124e-13, 4.6871939093494826e-11, -1.732711632957565e-10, -1.838870773833543e-15, 8.520119236071592e-16, 2.2896602469066618e-05, -2.039337700633091e-06, -1.6080524952894662e-08, 9.881854839768873e-10]

submit(Team_Key,list(vvtt))

'successfully submitted'